In [1]:
import numpy as np
import sympy as sp

In [8]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [10]:
sp.Matrix(np.array_split(criar_vetor_de_simbolos(4), 2))

Matrix([
[x1, x2],
[x3, x4]])

In [13]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [128]:
kronecker_binary([[1,0],[0,1]])

array([0, 1, 0, 0])

In [72]:
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(4), 2)))
variaveis

Matrix([
[x1*x3],
[x1*x4],
[x2*x3],
[x2*x4]])

In [16]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [162]:
numeros_decimais = [0,2,2,1]
vetores_binarios = []
tamanho_bits = int(len(numeros_decimais)/2)

for numero in numeros_decimais:
    try:
        vetor = decimal_para_binario_vetor(numero, tamanho_bits)
        vetores_binarios.append(vetor)
    except (TypeError, ValueError) as e:
        print(f"Erro ao converter o número {numero}: {e}")

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 1, 1, 0],
[0, 0, 0, 0]])

In [ ]:
regra = sp.Matrix(np.dot(vetores_binarios, kronecker_binary(np.array_split(criar_vetor_de_simbolos(4), 2)))) # expressões simbólicas
regra

Matrix([
[        x1*x3],
[        x2*x4],
[x1*x4 + x2*x3],
[            0]])

In [115]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(4)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = sp.lambdify(variaveis, expr)
    funcoes.append(func)

funcoes

[<function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>]

In [114]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(4)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = lambdastr(variaveis, expr)
    funcoes.append(func)

funcoes

['lambda x1,x2,x3,x4: (x1*x3)',
 'lambda x1,x2,x3,x4: (x2*x4)',
 'lambda x1,x2,x3,x4: (x1*x4 + x2*x3)',
 'lambda x1,x2,x3,x4: (0)']

In [ ]:
valores = np.array([decimal_para_binario_vetor(0,1),decimal_para_binario_vetor(0,1)]).flatten()
print(valores)

# Calcular os resultados
[f(*valores) for f in funcoes]

[1 0 1 0]


[1, 0, 0, 0]

In [175]:
sp.Matrix(np.dot(vetores_binarios, [0,1,0,0]))

Matrix([
[0],
[0],
[1],
[0]])

In [200]:
import itertools

# Usando produto cartesiano
combinacoes = list(itertools.product([0, 1], repeat=2))

for comb in combinacoes:
	# Converte cada bit da combinação para um vetor binário e achata em uma única dimensão
	valores = np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]).flatten()
	print(valores,[f(*valores) for f in funcoes])

[1 0 1 0] [1, 0, 0, 0]
[1 0 0 1] [0, 0, 1, 0]
[0 1 1 0] [0, 0, 1, 0]
[0 1 0 1] [0, 1, 0, 0]
